[Reference](https://medium.com/@adelbasli/beyond-the-label-decoding-food-processing-with-nova-and-python-5ff12334d63e)

In [1]:
import requests
import json

def get_nova_score(barcode):
    """
    Fetches the NOVA score for a given product barcode from Open Food Facts.
    """
    base_url = "https://world.openfoodfacts.net/api/v2/product/"
    url = f"{base_url}{barcode}"

    # It is good practice to send a User-Agent header to identify your application.
    # Replace 'your_email@example.com' with your actual contact email.
    headers = {
        "User-Agent": "MediumArticleNovaPython/1.0 (contact: your_email@example.com)"
    }

    try:
        # Make the GET request to the Open Food Facts API with a timeout.
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx).

        data = response.json()

        # Check if the product was found and contains NOVA information.
        if data.get('status') == 1 and 'product' in data:
            product_info = data['product']
            product_name = product_info.get('product_name', 'N/A')
            nova_group = product_info.get('nova_group') # This is the key field for the NOVA score.

            # Map the NOVA group number to its descriptive text.
            nova_group_description = {
                1: "Group 1: Unprocessed or Minimally Processed Foods",
                2: "Group 2: Processed Culinary Ingredients",
                3: "Group 3: Processed Foods",
                4: "Group 4: Ultra-Processed Food and Drink Products"
            }.get(nova_group, "NOVA score not available or unknown group")

            print(f"Product Name: {product_name}")
            print(f"Barcode: {barcode}")
            print(f"NOVA Group: {nova_group} - {nova_group_description}")
            return nova_group
        else:
            print(f"Product with barcode {barcode} not found or data incomplete.")
            print(f"API Response Status: {data.get('status_verbose', 'N/A')}")
            return None

    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error occurred: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting to the API: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Request timed out: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"An unexpected request error occurred: {err}")
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from response for barcode {barcode}.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None

# --- Example Usage ---
# You can find barcodes on the Open Food Facts website (world.openfoodfacts.org)
# or by scanning products in your pantry with the Open Food Facts app.
# The actual NOVA group for these examples depends on the data available in Open Food Facts.

print("\n--- Testing with Coca-Cola (common UPF, likely Group 4) ---")
get_nova_score("5449000000996")

print("\n--- Testing with Canned Tuna (common Processed Food, likely Group 3) ---")
get_nova_score("4056489254683")

print("\n--- Testing with Plain Yogurt (common Minimally Processed Food, likely Group 1) ---")
get_nova_score("4088600060880")

print("\n--- Testing with a non-existent barcode ---")
get_nova_score("9999999999999")


--- Testing with Coca-Cola (common UPF, likely Group 4) ---
Product Name: Coca-Cola 33cl
Barcode: 5449000000996
NOVA Group: 4 - Group 4: Ultra-Processed Food and Drink Products

--- Testing with Canned Tuna (common Processed Food, likely Group 3) ---
Product Name: Thunfisch Filets in eigenem Saft
Barcode: 4056489254683
NOVA Group: 3 - Group 3: Processed Foods

--- Testing with Plain Yogurt (common Minimally Processed Food, likely Group 1) ---
Product Name: 10% Fat Authentic Greek Yoghurt
Barcode: 4088600060880
NOVA Group: 1 - Group 1: Unprocessed or Minimally Processed Foods

--- Testing with a non-existent barcode ---
Product Name: N/A
Barcode: 9999999999999
NOVA Group: None - NOVA score not available or unknown group
